In [1]:
import pandas as pd
import json

# 從JSON文件中逐行讀取數據
with open('tweets_DM.json', 'r') as file:
    lines = file.readlines()

# 初始化一個空的列表
data_list = []

# 循環迭代每一行數據
for line in lines:
    data_dict = json.loads(line)
    score = data_dict["_score"]
    #index = data_dict["_index"]
    source = data_dict["_source"]
    tweet_id = source["tweet"]["tweet_id"]
    text = source["tweet"]["text"]
    hashtags = source["tweet"]["hashtags"] if "hashtags" in source["tweet"] else []
    #crawldate = data_dict["_crawldate"]
    #data_type = data_dict["_type"]

    # 將提取的字段添加到列表
    data_list.append({
        "tweet_id": tweet_id,
        "score": score,
        #"index": index,
        "text": text,
        "hashtags": hashtags,
        #"crawldate": crawldate,
        #"type": data_type
    })

# 一次建立 DataFrame
df = pd.DataFrame(data_list)



In [2]:
df.head()

,tweet_id,score,text,hashtags
0,0x376b20,391,"People who post ""add me on #Snapchat"" must be ...",[Snapchat]
1,0x2d5350,433,"@brianklaas As we see, Trump is dangerous to #...","[freepress, TrumpLegacy, CNN]"
2,0x28b412,232,"Confident of your obedience, I write to you, k...",[bibleverse]
3,0x1cd5b0,376,Now ISSA is stalking Tasha 😂😂😂 <LH>,[]
4,0x2de201,989,"""Trust is not the same as faith. A friend is s...",[]


In [3]:
# 讀取包含train或test標籤的CSV文件
labels_df = pd.read_csv('data_identification.csv')
print(labels_df['identification'].unique())

['test' 'train']


In [4]:

# 讀取包含情緒標籤的CSV文件
sentiments_df = pd.read_csv('emotion.csv')
print(sentiments_df['emotion'].unique())


['sadness' 'disgust' 'anticipation' 'joy' 'trust' 'anger' 'fear'
 'surprise']


In [5]:
#以tweet_id為準，將情緒標籤合併
merged_df = labels_df.merge(sentiments_df, on='tweet_id', how='outer').merge(df, on='tweet_id', how='outer')
merged_df.head()

,tweet_id,identification,emotion,score,text,hashtags
0,0x28cc61,test,NaN,107,@Habbo I've seen two separate colours of the e...,[]
1,0x29e452,train,joy,809,Huge Respect🖒 @JohnnyVegasReal talking about l...,[]
2,0x2b3819,train,joy,808,Yoooo we hit all our monthly goals with the ne...,"[spateradio, app]"
3,0x2db41f,test,NaN,728,@FoxNews @KellyannePolls No serious self respe...,[]
4,0x2a2acc,train,trust,16,@KIDSNTS @PICU_BCH @uhbcomms @BWCHBoss Well do...,[]


In [6]:
#分出訓練資料
train_df = merged_df[merged_df['identification'] == 'train']
print("Train DataFrame:")
print(train_df.head())


Train DataFrame:
   tweet_id identification       emotion  score  \
1  0x29e452          train           joy    809   
2  0x2b3819          train           joy    808   
4  0x2a2acc          train         trust     16   
5  0x2a8830          train           joy    768   
6  0x20b21d          train  anticipation     70   

                                                text  \
1  Huge Respect🖒 @JohnnyVegasReal talking about l...   
2  Yoooo we hit all our monthly goals with the ne...   
4  @KIDSNTS @PICU_BCH @uhbcomms @BWCHBoss Well do...   
5  Come join @ambushman27 on #PUBG while he striv...   
6  @fanshixieen2014 Blessings!My #strength little...   

                                            hashtags  
1                                                 []  
2                                  [spateradio, app]  
4                                                 []  
5  [PUBG, GamersUnite, twitch, BeHealthy, StayPos...  
6                             [strength, bones, God]  


In [7]:
#分出測試資料
test_df = merged_df[merged_df['identification'] == 'test']
print("Test DataFrame:")
print(test_df[300:305])

Test DataFrame:
      tweet_id identification emotion  score  \
1366  0x231381           test     NaN    218   
1368  0x24e2ce           test     NaN    819   
1373  0x1dc3a5           test     NaN     76   
1375  0x2895fa           test     NaN    761   
1377  0x3020b7           test     NaN     46   

                                                   text   hashtags  
1366  @HMpoll @king_My18 Yeah. And when a fan of bot...         []  
1368  What is #Chelsea doing? Selling one of the bes...  [Chelsea]  
1373  Big things coming soon for me a my little fam ...         []  
1375  Woke up this morning watchd a video of someone...         []  
1377  Credit card reminds me of the success I get in...         []  


In [8]:
#Feature engineering 的部分我使用Bag of words
from sklearn.feature_extraction.text import CountVectorizer
import nltk
BOW_vectorizer = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize) 
BOW_vectorizer.fit(train_df['text'])

train_data_BOW_features = BOW_vectorizer.transform(train_df['text'])
test_data_BOW_features = BOW_vectorizer.transform(test_df['text'])

c:\Users\user\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [16]:
#在訓練模型前，我把特徵值先進行標準化，以達到更好的performence
#因為硬體限制，我使用較為簡單的Logistic Regression，可達到準確率0.388
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=False)
X_train_vectorized_scaled = scaler.fit_transform(train_data_BOW_features)
X_test_vectorized_scaled = scaler.transform(test_data_BOW_features)
      
y_train = train_df['emotion']

model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

model.fit(X_train_vectorized_scaled, y_train)
y_pred = model.predict(X_test_vectorized_scaled)
test_df['emotion'] = y_pred
test_df = test_df.rename(columns={'tweet_id': 'id'})
header = ["id", "emotion"]
test_df.to_csv('output.csv', columns=header, index = False)
test_df.head()

c:\Users\user\miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,id,identification,emotion,score,text,hashtags
0,0x28cc61,test,joy,107,@Habbo I've seen two separate colours of the e...,[]
1,0x2db41f,test,joy,728,@FoxNews @KellyannePolls No serious self respe...,[]
2,0x2466f6,test,joy,491,"Looking for a new car, and it says 1 lady owne...",[womendrivers]
3,0x23f9e9,test,joy,28,@cineworld “only the brave” just out and fount...,[robbingmembers]
4,0x1fb4e1,test,disgust,925,Felt like total dog 💩 going into open gym and ...,[]
